# Built upon my Max's code and my own code from hmwk 2 in order to create the structure for trailing stops.

Process:
1. Edit the volatility table and call it ts (trailing stops). Add a column for returns.
2. Add a results table which keeps track of the balance in our investements and the profit/loss.
3. Initialise trading with a funciton that assigns 100 unit positions to 10 currencies. 5 long positions and 5 short positions.
4. Run the trailing stop strategy as outlined in the assignment instructions:
    a. Check each hour to see if our positions are within an acceptable loss making range. If they have loss more than acceptable for that period, exit the position.
    b. If the position has not loss as much as the acceptable amount (or even made money!) than add a further 100 units to the investment.
5. Exit after 10 hours.

In [1]:
# Import required libraries
import datetime
import time
from polygon import RESTClient
from sqlalchemy import create_engine 
from sqlalchemy import text
import pandas as pd
from math import sqrt
from math import isnan
import matplotlib.pyplot as plt
from numpy import mean
from numpy import std
from math import floor
import numpy as np

In [2]:
# The following 10 blocks of code define the classes for storing the the return data, for each
# currency pair.
        
# Define the AUDUSD_return class - each instance will store one row from the dataframe
class AUDUSD_return(object):
    # Variable to store the total number of instantiated objects in this class
    num = 0
    # Variable to store the running sum of the return
    run_sum = 0
    run_squared_sum = 0
    run_sum_of_std = 0
    last_price = -1
    
    # Init all the necessary variables when instantiating the class
    def __init__(self, tick_time, avg_price):
        
        # Store each column value into a variable in the class instance
        self.tick_time = tick_time
        #self.price = avg_price
        
        if AUDUSD_return.last_price == -1:
            hist_return = float('NaN')
        else:
            hist_return = (avg_price - AUDUSD_return.last_price) / AUDUSD_return.last_price
        
        self.hist_return = hist_return
        if isnan(hist_return):
            AUDUSD_return.run_sum = 0
        else:
            # Increment the counter
            if AUDUSD_return.num < 5:
                AUDUSD_return.num += 1
            AUDUSD_return.run_sum += hist_return
        AUDUSD_return.last_price = avg_price
        
    def add_to_running_squared_sum(self,avg):
        if isnan(self.hist_return) == False:
            AUDUSD_return.run_squared_sum += (self.hist_return - avg)**2
    
    def get_avg(self,pop_value):
        if isnan(self.hist_return) == False:
            AUDUSD_return.run_sum -= pop_value
            avg = AUDUSD_return.run_sum/(AUDUSD_return.num)
            self.avg_return = avg
            return avg
    
    def get_std(self):
        if isnan(self.hist_return) == False:
            std = sqrt(AUDUSD_return.run_squared_sum/(AUDUSD_return.num))
            self.std_return = std
            AUDUSD_return.run_sum_of_std += std
            AUDUSD_return.run_squared_sum = 0
            return std
    
    def get_avg_std(self,pop_value):
        if isnan(self.hist_return) == False:
            AUDUSD_return.run_sum_of_std -= pop_value
            avg_std = AUDUSD_return.run_sum_of_std/(AUDUSD_return.num)
            self.avg_of_std_return = avg_std 
            return avg_std

In [3]:
# Define the GBPEUR_return class - each instance will store one row from the dataframe
class GBPEUR_return(object):
    # Variable to store the total number of instantiated objects in this class
    num = 0
    # Variable to store the running sum of the return
    run_sum = 0
    run_squared_sum = 0
    run_sum_of_std = 0
    last_price = -1
    
    # Init all the necessary variables when instantiating the class
    def __init__(self, tick_time, avg_price):
        
        # Store each column value into a variable in the class instance
        self.tick_time = tick_time
        #self.price = avg_price
        
        if GBPEUR_return.last_price == -1:
            hist_return = float('NaN')
        else:
            hist_return = (avg_price - GBPEUR_return.last_price) / GBPEUR_return.last_price
        
        self.hist_return = hist_return
        if isnan(hist_return):
            GBPEUR_return.run_sum = 0
        else:
            # Increment the counter
            if GBPEUR_return.num < 5:
                GBPEUR_return.num += 1
            GBPEUR_return.run_sum += hist_return
        GBPEUR_return.last_price = avg_price
        
    def add_to_running_squared_sum(self,avg):
        if isnan(self.hist_return) == False:
            GBPEUR_return.run_squared_sum += (self.hist_return - avg)**2
    
    def get_avg(self,pop_value):
        if isnan(self.hist_return) == False:
            GBPEUR_return.run_sum -= pop_value
            avg = GBPEUR_return.run_sum/(GBPEUR_return.num)
            self.avg_return = avg
            return avg
    
    def get_std(self):
        if isnan(self.hist_return) == False:
            std = sqrt(GBPEUR_return.run_squared_sum/(GBPEUR_return.num))
            self.std_return = std
            GBPEUR_return.run_sum_of_std += std
            GBPEUR_return.run_squared_sum = 0
            return std
    
    def get_avg_std(self,pop_value):
        if isnan(self.hist_return) == False:
            GBPEUR_return.run_sum_of_std -= pop_value
            avg_std = GBPEUR_return.run_sum_of_std/(GBPEUR_return.num)
            self.avg_of_std_return = avg_std 
            return avg_std

In [4]:
# Define the USDCAD_return class - each instance will store one row from the dataframe
class USDCAD_return(object):
    # Variable to store the total number of instantiated objects in this class
    num = 0
    # Variable to store the running sum of the return
    run_sum = 0
    run_squared_sum = 0
    run_sum_of_std = 0
    last_price = -1
    
    # Init all the necessary variables when instantiating the class
    def __init__(self, tick_time, avg_price):

        # Store each column value into a variable in the class instance
        self.tick_time = tick_time
        #self.price = avg_price
        
        if USDCAD_return.last_price == -1:
            hist_return = float('NaN')
        else:
            hist_return = (avg_price - USDCAD_return.last_price) / USDCAD_return.last_price
        
        self.hist_return = hist_return
        if isnan(hist_return):
            USDCAD_return.run_sum = 0
        else:
            # Increment the counter
            if USDCAD_return.num < 5:
                USDCAD_return.num += 1
            USDCAD_return.run_sum += hist_return
        USDCAD_return.last_price = avg_price
        
    def add_to_running_squared_sum(self,avg):
        if isnan(self.hist_return) == False:
            USDCAD_return.run_squared_sum += (self.hist_return - avg)**2
    
    def get_avg(self,pop_value):
        if isnan(self.hist_return) == False:
            USDCAD_return.run_sum -= pop_value
            avg = USDCAD_return.run_sum/(USDCAD_return.num)
            self.avg_return = avg
            return avg
    
    def get_std(self):
        if isnan(self.hist_return) == False:
            std = sqrt(USDCAD_return.run_squared_sum/(USDCAD_return.num))
            self.std_return = std
            USDCAD_return.run_sum_of_std += std
            USDCAD_return.run_squared_sum = 0
            return std
    
    def get_avg_std(self,pop_value):
        if isnan(self.hist_return) == False:
            USDCAD_return.run_sum_of_std -= pop_value
            avg_std = USDCAD_return.run_sum_of_std/(USDCAD_return.num)
            self.avg_of_std_return = avg_std 
            return avg_std

In [5]:
# Define the USDJPY_return class - each instance will store one row from the dataframe
class USDJPY_return(object):
    # Variable to store the total number of instantiated objects in this class
    num = 0
    # Variable to store the running sum of the return
    run_sum = 0
    run_squared_sum = 0
    run_sum_of_std = 0
    last_price = -1
    
    # Init all the necessary variables when instantiating the class
    def __init__(self, tick_time, avg_price):
        
        # Store each column value into a variable in the class instance
        self.tick_time = tick_time
        #self.price = avg_price
        
        if USDJPY_return.last_price == -1:
            hist_return = float('NaN')
        else:
            hist_return = (avg_price - USDJPY_return.last_price) / USDJPY_return.last_price
        
        self.hist_return = hist_return
        if isnan(hist_return):
            USDJPY_return.run_sum = 0
        else:
            # Increment the counter
            if USDJPY_return.num < 5:
                USDJPY_return.num += 1
            USDJPY_return.run_sum += hist_return
        USDJPY_return.last_price = avg_price
        
    def add_to_running_squared_sum(self,avg):
        if isnan(self.hist_return) == False:
            USDJPY_return.run_squared_sum += (self.hist_return - avg)**2
    
    def get_avg(self,pop_value):
        if isnan(self.hist_return) == False:
            USDJPY_return.run_sum -= pop_value
            avg = USDJPY_return.run_sum/(USDJPY_return.num)
            self.avg_return = avg
            return avg
    
    def get_std(self):
        if isnan(self.hist_return) == False:
            std = sqrt(USDJPY_return.run_squared_sum/(USDJPY_return.num))
            self.std_return = std
            USDJPY_return.run_sum_of_std += std
            USDJPY_return.run_squared_sum = 0
            return std
    
    def get_avg_std(self,pop_value):
        if isnan(self.hist_return) == False:
            USDJPY_return.run_sum_of_std -= pop_value
            avg_std = USDJPY_return.run_sum_of_std/(USDJPY_return.num)
            self.avg_of_std_return = avg_std 
            return avg_std

In [6]:
# Define the USDMXN_return class - each instance will store one row from the dataframe
class USDMXN_return(object):
    # Variable to store the total number of instantiated objects in this class
    num = 0
    # Variable to store the running sum of the return
    run_sum = 0
    run_squared_sum = 0
    run_sum_of_std = 0
    last_price = -1
    
    # Init all the necessary variables when instantiating the class
    def __init__(self, tick_time, avg_price):
        
        # Store each column value into a variable in the class instance
        self.tick_time = tick_time
        #self.price = avg_price
        
        if USDMXN_return.last_price == -1:
            hist_return = float('NaN')
        else:
            hist_return = (avg_price - USDMXN_return.last_price) / USDMXN_return.last_price
        
        self.hist_return = hist_return
        if isnan(hist_return):
            USDMXN_return.run_sum = 0
        else:
            # Increment the counter
            if USDMXN_return.num < 5:
                USDMXN_return.num += 1
            USDMXN_return.run_sum += hist_return
        USDMXN_return.last_price = avg_price
        
    def add_to_running_squared_sum(self,avg):
        if isnan(self.hist_return) == False:
            USDMXN_return.run_squared_sum += (self.hist_return - avg)**2
    
    def get_avg(self,pop_value):
        if isnan(self.hist_return) == False:
            USDMXN_return.run_sum -= pop_value
            avg = USDMXN_return.run_sum/(USDMXN_return.num)
            self.avg_return = avg
            return avg
    
    def get_std(self):
        if isnan(self.hist_return) == False:
            std = sqrt(USDMXN_return.run_squared_sum/(USDMXN_return.num))
            self.std_return = std
            USDMXN_return.run_sum_of_std += std
            USDMXN_return.run_squared_sum = 0
            return std
    
    def get_avg_std(self,pop_value):
        if isnan(self.hist_return) == False:
            USDMXN_return.run_sum_of_std -= pop_value
            avg_std = USDMXN_return.run_sum_of_std/(USDMXN_return.num)
            self.avg_of_std_return = avg_std 
            return avg_std

In [7]:
# Define the EURUSD_return class - each instance will store one row from the dataframe
class EURUSD_return(object):
    # Variable to store the total number of instantiated objects in this class
    num = 0
    # Variable to store the running sum of the return
    run_sum = 0
    run_squared_sum = 0
    run_sum_of_std = 0
    last_price = -1
    
    # Init all the necessary variables when instantiating the class
    def __init__(self, tick_time, avg_price):
        
        # Store each column value into a variable in the class instance
        self.tick_time = tick_time
        #self.price = avg_price
        
        if EURUSD_return.last_price == -1:
            hist_return = float('NaN')
        else:
            hist_return = (avg_price - EURUSD_return.last_price) / EURUSD_return.last_price
        
        self.hist_return = hist_return
        if isnan(hist_return):
            EURUSD_return.run_sum = 0
        else:
            # Increment the counter
            if EURUSD_return.num < 5:
                EURUSD_return.num += 1
            EURUSD_return.run_sum += hist_return
        EURUSD_return.last_price = avg_price
        
    def add_to_running_squared_sum(self,avg):
        if isnan(self.hist_return) == False:
            EURUSD_return.run_squared_sum += (self.hist_return - avg)**2
    
    def get_avg(self,pop_value):
        if isnan(self.hist_return) == False:
            EURUSD_return.run_sum -= pop_value
            avg = EURUSD_return.run_sum/(EURUSD_return.num)
            self.avg_return = avg
            return avg
    
    def get_std(self):
        if isnan(self.hist_return) == False:
            std = sqrt(EURUSD_return.run_squared_sum/(EURUSD_return.num))
            self.std_return = std
            EURUSD_return.run_sum_of_std += std
            EURUSD_return.run_squared_sum = 0
            return std
    
    def get_avg_std(self,pop_value):
        if isnan(self.hist_return) == False:
            EURUSD_return.run_sum_of_std -= pop_value
            avg_std = EURUSD_return.run_sum_of_std/(EURUSD_return.num)
            self.avg_of_std_return = avg_std 
            return avg_std

In [8]:
# Define the USDCNY_return class - each instance will store one row from the dataframe
class USDCNY_return(object):
    # Variable to store the total number of instantiated objects in this class
    num = 0
    # Variable to store the running sum of the return
    run_sum = 0
    run_squared_sum = 0
    run_sum_of_std = 0
    last_price = -1
    
    # Init all the necessary variables when instantiating the class
    def __init__(self, tick_time, avg_price):
        
        # Store each column value into a variable in the class instance
        self.tick_time = tick_time
        #self.price = avg_price
        
        if USDCNY_return.last_price == -1:
            hist_return = float('NaN')
        else:
            hist_return = (avg_price - USDCNY_return.last_price) / USDCNY_return.last_price
        
        self.hist_return = hist_return
        if isnan(hist_return):
            USDCNY_return.run_sum = 0
        else:
            # Increment the counter
            if USDCNY_return.num < 5:
                USDCNY_return.num += 1
            USDCNY_return.run_sum += hist_return
        USDCNY_return.last_price = avg_price
        
    def add_to_running_squared_sum(self,avg):
        if isnan(self.hist_return) == False:
            USDCNY_return.run_squared_sum += (self.hist_return - avg)**2
    
    def get_avg(self,pop_value):
        if isnan(self.hist_return) == False:
            USDCNY_return.run_sum -= pop_value
            avg = USDCNY_return.run_sum/(USDCNY_return.num)
            self.avg_return = avg
            return avg
    
    def get_std(self):
        if isnan(self.hist_return) == False:
            std = sqrt(USDCNY_return.run_squared_sum/(USDCNY_return.num))
            self.std_return = std
            USDCNY_return.run_sum_of_std += std
            USDCNY_return.run_squared_sum = 0
            return std
    
    def get_avg_std(self,pop_value):
        if isnan(self.hist_return) == False:
            USDCNY_return.run_sum_of_std -= pop_value
            avg_std = USDCNY_return.run_sum_of_std/(USDCNY_return.num)
            self.avg_of_std_return = avg_std 
            return avg_std

In [9]:
# Define the USDCZK_return class - each instance will store one row from the dataframe
class USDCZK_return(object):
    # Variable to store the total number of instantiated objects in this class
    num = 0
    # Variable to store the running sum of the return
    run_sum = 0
    run_squared_sum = 0
    run_sum_of_std = 0
    last_price = -1
    
    # Init all the necessary variables when instantiating the class
    def __init__(self, tick_time, avg_price):
        
        # Store each column value into a variable in the class instance
        self.tick_time = tick_time
        #self.price = avg_price
        
        if USDCZK_return.last_price == -1:
            hist_return = float('NaN')
        else:
            hist_return = (avg_price - USDCZK_return.last_price) / USDCZK_return.last_price
        
        self.hist_return = hist_return
        if isnan(hist_return):
            USDCZK_return.run_sum = 0
        else:
            # Increment the counter
            if USDCZK_return.num < 5:
                USDCZK_return.num += 1            
            USDCZK_return.run_sum += hist_return
        USDCZK_return.last_price = avg_price
        
    def add_to_running_squared_sum(self,avg):
        if isnan(self.hist_return) == False:
            USDCZK_return.run_squared_sum += (self.hist_return - avg)**2
    
    def get_avg(self,pop_value):
        if isnan(self.hist_return) == False:
            USDCZK_return.run_sum -= pop_value
            avg = USDCZK_return.run_sum/(USDCZK_return.num)
            self.avg_return = avg
            return avg
    
    def get_std(self):
        if isnan(self.hist_return) == False:
            std = sqrt(USDCZK_return.run_squared_sum/(USDCZK_return.num))
            self.std_return = std
            USDCZK_return.run_sum_of_std += std
            USDCZK_return.run_squared_sum = 0
            return std
    
    def get_avg_std(self,pop_value):
        if isnan(self.hist_return) == False:
            USDCZK_return.run_sum_of_std -= pop_value
            avg_std = USDCZK_return.run_sum_of_std/(USDCZK_return.num)
            self.avg_of_std_return = avg_std 
            return avg_std

In [10]:
# Define the USDPLN_return class - each instance will store one row from the dataframe
class USDPLN_return(object):
    # Variable to store the total number of instantiated objects in this class
    num = 0
    # Variable to store the running sum of the return
    run_sum = 0
    run_squared_sum = 0
    run_sum_of_std = 0
    last_price = -1
    
    # Init all the necessary variables when instantiating the class
    def __init__(self, tick_time, avg_price):
        
        # Store each column value into a variable in the class instance
        self.tick_time = tick_time
        #self.price = avg_price
        
        if USDPLN_return.last_price == -1:
            hist_return = float('NaN')
        else:
            hist_return = (avg_price - USDPLN_return.last_price) / USDPLN_return.last_price
        
        self.hist_return = hist_return
        if isnan(hist_return):
            USDPLN_return.run_sum = 0
        else:
            # Increment the counter
            if USDPLN_return.num < 5:
                USDPLN_return.num += 1
            USDPLN_return.run_sum += hist_return
        USDPLN_return.last_price = avg_price
        
    def add_to_running_squared_sum(self,avg):
        if isnan(self.hist_return) == False:
            USDPLN_return.run_squared_sum += (self.hist_return - avg)**2
    
    def get_avg(self,pop_value):
        if isnan(self.hist_return) == False:
            USDPLN_return.run_sum -= pop_value
            avg = USDPLN_return.run_sum/(USDPLN_return.num)
            self.avg_return = avg
            return avg
    
    def get_std(self):
        if isnan(self.hist_return) == False:
            std = sqrt(USDPLN_return.run_squared_sum/(USDPLN_return.num))
            self.std_return = std
            USDPLN_return.run_sum_of_std += std
            USDPLN_return.run_squared_sum = 0
            return std
    
    def get_avg_std(self,pop_value):
        if isnan(self.hist_return) == False:
            USDPLN_return.run_sum_of_std -= pop_value
            avg_std = USDPLN_return.run_sum_of_std/(USDPLN_return.num)
            self.avg_of_std_return = avg_std 
            return avg_std

In [11]:
# Define the USDINR_return class - each instance will store one row from the dataframe
class USDINR_return(object):
    # Variable to store the total number of instantiated objects in this class
    num = 0
    # Variable to store the running sum of the return
    run_sum = 0
    run_squared_sum = 0
    run_sum_of_std = 0
    last_price = -1
    
    # Init all the necessary variables when instantiating the class
    def __init__(self, tick_time, avg_price):
        
        # Store each column value into a variable in the class instance
        self.tick_time = tick_time
        #self.price = avg_price
        
        if USDINR_return.last_price == -1:
            hist_return = float('NaN')
        else:
            hist_return = (avg_price - USDINR_return.last_price) / USDINR_return.last_price
        
        self.hist_return = hist_return
        if isnan(hist_return):
            USDINR_return.run_sum = 0
        else:
            # Increment the counter
            if USDINR_return.num < 5:
                USDINR_return.num += 1
            USDINR_return.run_sum += hist_return
        USDINR_return.last_price = avg_price
    
    def add_to_running_squared_sum(self,avg):
        if isnan(self.hist_return) == False:
            USDINR_return.run_squared_sum += (self.hist_return - avg)**2
    
    def get_avg(self,pop_value):
        if isnan(self.hist_return) == False:
            USDINR_return.run_sum -= pop_value
            avg = USDINR_return.run_sum/(USDINR_return.num)
            self.avg_return = avg
            return avg
    
    def get_std(self):
        if isnan(self.hist_return) == False:
            std = sqrt(USDINR_return.run_squared_sum/(USDINR_return.num))
            self.std_return = std
            USDINR_return.run_sum_of_std += std
            USDINR_return.run_squared_sum = 0
            return std
    
    def get_avg_std(self,pop_value):
        if isnan(self.hist_return) == False:
            USDINR_return.run_sum_of_std -= pop_value
            avg_std = USDINR_return.run_sum_of_std/(USDINR_return.num)
            self.avg_of_std_return = avg_std 
            return avg_std

In [12]:
# We can buy, sell, or do nothing each time we make a decision.
# This class defies a nobject for keeping track of our current investments/profits for each currency pair
class portfolio(object):
    def __init__(self,from_,to):
        # Initialize the 'From' currency amont to 1
        self.amount = 1
        self.curr2 = 0
        self.from_ = from_
        self.to = to
        # We want to keep track of state, to see what our next trade should be
        self.Prev_Action_was_Buy = False
    
    # This defines a function to buy the 'To' currency. It will always buy the max amount, in whole number
    # increments
    def buy_curr(self, price):
        if self.amount >= 1:
            num_to_buy = floor(self.amount)
            self.amount -= num_to_buy
            self.Prev_Action_was_Buy = True
            self.curr2 += num_to_buy*price
            print("Bought %d worth of the target currency (%s). Our current profits and losses in the original currency (%s) are: %f." % (num_to_buy,self.to,self.from_,(self.amount-1)))
        else:
            print("There was not enough of the original currency (%s) to make another buy." % self.from_)
    # This defines a function to sell the 'To' currency. It will always sell the max amount, in a whole number
    # increments
    def sell_curr(self, price):
        if self.curr2 >= 1:
            num_to_sell = floor(self.curr2)
            self.amount += num_to_sell * (1/price)
            self.Prev_Action_was_Buy = False
            self.curr2 -= num_to_sell
            print("Sold %d worth of the target currency (%s). Our current profits and losses in the original currency (%s) are: %f." % (num_to_sell,self.to,self.from_,(self.amount-1)))
        else:
            print("There was not enough of the target currency (%s) to make another sell." % self.to)   

In [13]:
# Function slightly modified from polygon sample code to format the date string 
def ts_to_datetime(ts) -> str:
    return datetime.datetime.fromtimestamp(ts / 1000.0).strftime('%Y-%m-%d %H:%M:%S')

# Function which clears the raw data tables once we have aggregated the data in a 6 minute interval
def reset_raw_data_tables(engine,currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            conn.execute(text("DROP TABLE "+curr[0]+curr[1]+"_raw;"))
            conn.execute(text("CREATE TABLE "+curr[0]+curr[1]+"_raw(ticktime text, fxrate  numeric, inserttime text);"))

# This creates a table for storing the raw, unaggregated price data for each currency pair in the SQLite database
def initialize_raw_data_tables(engine,currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            conn.execute(text("CREATE TABLE "+curr[0]+curr[1]+"_raw(ticktime text, fxrate  numeric, inserttime text);"))

# This creates a table for storing the (6 min interval) aggregated price data for each currency pair in the SQLite database            
def initialize_aggregated_tables(engine,currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            conn.execute(text("CREATE TABLE "+curr[0]+curr[1]+"_agg(inserttime text, avgfxrate  numeric, stdfxrate numeric);"))

# Create a table for storing data required for hmwk2 + 3
def intialize_ts_tables(engine,currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            conn.execute(text("CREATE TABLE "+curr[0]+curr[1]+"_ts(period text, maximum numeric, minimum numeric, mean numeric, volatilty numeric, fd numeric, return numeric);"))

# Create a table for storing the results from the stop loss strategy
def intialize_results_tables(engine,currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            conn.execute(text("CREATE TABLE "+curr[0]+curr[1]+"_results(period text, position text, balance numeric, profitloss numeric, status text);"))
            
            
# This function is called every 6 minutes to aggregate the data, store it in the aggregate table, 
# and then delete the raw data
def aggregate_raw_data_tables(engine,currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            result = conn.execute(text("SELECT AVG(fxrate) as avg_price, COUNT(fxrate) as tot_count FROM "+curr[0]+curr[1]+"_raw;"))
            for row in result:
                avg_price = row.avg_price
                tot_count = row.tot_count
            std_res = conn.execute(text("SELECT SUM((fxrate - "+str(avg_price)+")*(fxrate - "+str(avg_price)+"))/("+str(tot_count)+"-1) as std_price FROM "+curr[0]+curr[1]+"_raw;"))
            for row in std_res:
                std_price = sqrt(row.std_price)
            date_res = conn.execute(text("SELECT MAX(ticktime) as last_date FROM "+curr[0]+curr[1]+"_raw;"))
            for row in date_res:
                last_date = row.last_date
            conn.execute(text("INSERT INTO "+curr[0]+curr[1]+"_agg (inserttime, avgfxrate, stdfxrate) VALUES (:inserttime, :avgfxrate, :stdfxrate);"),[{"inserttime": last_date, "avgfxrate": avg_price, "stdfxrate": std_price}])
            
            # This calculates and stores the return values
            exec("curr[2].append("+curr[0]+curr[1]+"_return(last_date,avg_price))")
            #exec("print(\"The return for "+curr[0]+curr[1]+" is:"+str(curr[2][-1].hist_return)+" \")")
            
            if len(curr[2]) > 5:
                try:
                    avg_pop_value = curr[2][-6].hist_return
                except:
                    avg_pop_value = 0
                if isnan(avg_pop_value) == True:
                    avg_pop_value = 0
            else:
                avg_pop_value = 0
            # Calculate the average return value and print it/store it
            curr_avg = curr[2][-1].get_avg(avg_pop_value)
            #exec("print(\"The average return for "+curr[0]+curr[1]+" is:"+str(curr_avg)+" \")")
            
            # Now that we have the average return, loop through the last 5 rows in the list to start compiling the 
            # data needed to calculate the standard deviation
            for row in curr[2][-5:]:
                row.add_to_running_squared_sum(curr_avg)
            
            # Calculate the standard dev using the avg
            curr_std = curr[2][-1].get_std()
            #exec("print(\"The standard deviation of the return for "+curr[0]+curr[1]+" is:"+str(curr_std)+" \")")
            
            # Calculate the average standard dev
            if len(curr[2]) > 5:
                try:
                    pop_value = curr[2][-6].std_return
                except:
                    pop_value = 0
            else:
                pop_value = 0
            curr_avg_std = curr[2][-1].get_avg_std(pop_value)
            #exec("print(\"The average standard deviation of the return for "+curr[0]+curr[1]+" is:"+str(curr_avg_std)+" \")")
            
#             # -------------------Investment Strategy-----------------------------------------------
#             try:
#                 return_value = curr[2][-1].hist_return
#             except:
#                 return_value = 0
#             if isnan(return_value) == True:
#                 return_value = 0

#             try:
#                 return_value_1 = curr[2][-2].hist_return
#             except:
#                 return_value_1 = 0
#             if isnan(return_value_1) == True:
#                 return_value_1 = 0

#             try:
#                 return_value_2 = curr[2][-3].hist_return
#             except:
#                 return_value_2 = 0
#             if isnan(return_value_2) == True:
#                 return_value_2 = 0

#             try:
#                 upp_band = curr[2][-1].avg_return + (1.5 * curr[2][-1].std_return)
#                 if return_value >= upp_band and curr[3].Prev_Action_was_Buy == True and return_value != 0: #  (return_value > 0) and (return_value_1 > 0) and   
#                     curr[3].sell_curr(avg_price)
#             except:
#                 pass

#             try:
#                 loww_band = curr[2][-1].avg_return - (1.5 * curr[2][-1].std_return)
#                 if return_value <= loww_band and curr[3].Prev_Action_was_Buy == False and return_value != 0: # and  (return_value < 0) and (return_value_1 < 0)
#                     curr[3].buy_curr(avg_price)
#             except:
#                 pass





#                    ----------------- Hmwk 3 -----------------------


# This funtion initialises trading by assigning 5 currencies long positions and 5 currencies short positions.
# It adds 100 to their balance.

def initialise_trading(engine, currency_pairs, dic):
    i = 0
    for currency in currency_pairs:
        # Set the input variables to the API
        from_ = currency[0]
        to = currency[1]
        i +=1
        
        if (i % 2 != 0):
            dic[f'{from_}{to} position'] = 'long'
            dic[f'{from_}{to} balance'] = 100
            dic[f'{from_}{to} trade_status'] = 'live'
            
        
        if (i % 2 == 0):
            dic[f'{from_}{to} position'] = 'short'
            dic[f'{from_}{to} balance'] = 100
            dic[f'{from_}{to} trade_status'] = 'live'
            

# This function contains the stop loss strategy as outlined in the assignment 
            
def stop_loss_strategy(engine, currency_pairs, dic, hours_past):
    
    # This function gets called every hour

    # Loop through each currency pair
    for currency in currency_pairs:
        # Set the input variables to the API
        from_ = currency[0]
        to = currency[1]
        
        
    
    # We can reference the acceptable loss from this logic array. This refactoring saved many lines of copy and pasted code.
    # TEach element is an acceptable loss, it's index + 1 is its corresponding phase. I.e 0.250 is acceptable in the first hour, 0.100 is acceptable in the second hour.
    
        logic = [0.250, 0.150, 0.100, 0.050, 0.050, 0.050, 0.050, 0.050, 0.050, 0.050]
            
               
        # If 10 hours have past, exit the trades and compute the profit/loss
        if hours_past == 10:
            # Exit trades and compute balance, profit or loss
            dic[f'{from_}{to} trade_status'] = 'exited'
            dic[f'{from_}{to} balance'] = dic[f'{from_}{to} balance'] * (1 - dic[f'{from_}{to} return'])
            
            if (dic[f'{from_}{to} position'] == 'long'):
                dic[f'{from_}{to} profit_loss'] = (dic[f'{from_}{to} balance'] - dic[f'{from_}{to} total_invested'])
            
            if (dic[f'{from_}{to} position'] == 'short'):
                dic[f'{from_}{to} profit_loss'] = -(dic[f'{from_}{to} balance'] - dic[f'{from_}{to} total_invested'])
                
            
                
        else:
            # Follow the trading logic   
            for index, val in enumerate(logic):
                #If first period
                if hours_past == index+1:
                    #If long
                    if (dic[f'{from_}{to} position'] == 'long') & (dic[f'{from_}{to} trade_status'] == 'live'):
                    
                        # If long and loss is larger than accepted loss compute loss and close trade
                        if ((dic[f'{from_}{to} return']) < -(logic[index]/100)):
                            dic[f'{from_}{to} trade_status'] = 'exited'
                            dic[f'{from_}{to} balance'] = dic[f'{from_}{to} balance'] * (1 - dic[f'{from_}{to} return'])
                            

                        # If long and loss is less than accepted loss, compute profit or loss, add it to the position and add another 100 to the trade 
                        if ((dic[f'{from_}{to} return']) >= -(logic[index]/100)):
                            dic[f'{from_}{to} balance'] = dic[f'{from_}{to} balance'] + (dic[f'{from_}{to} balance'] * dic[f'{from_}{to} return']) + 100
                            dic[f'{from_}{to} total_invested'] += 100
                        
                        dic[f'{from_}{to} profit_loss'] = (dic[f'{from_}{to} balance'] - dic[f'{from_}{to} total_invested'])
                        
                        
                    if (dic[f'{from_}{to} position'] == 'short') & (dic[f'{from_}{to} trade_status'] == 'live'):
                         # If short and gain is larger than accepted gain compute loss and close trade
                        if ((dic[f'{from_}{to} return']) > (logic[index]/100)):
                            dic[f'{from_}{to} trade_status'] = 'exited'
                            dic[f'{from_}{to} balance'] = dic[f'{from_}{to} balance'] * (1 - dic[f'{from_}{to} return'])
                            
                         # If short and gain is less than accepted gain compute profit or loss add it to the position and add another 100 to the trade
                        if ((dic[f'{from_}{to} return']) <= (logic[index]/100)):
                            dic[f'{from_}{to} balance'] = dic[f'{from_}{to} balance'] + (dic[f'{from_}{to} balance'] * dic[f'{from_}{to} return']) + 100
                            dic[f'{from_}{to} total_invested'] += 100

                        dic[f'{from_}{to} profit_loss'] = -(dic[f'{from_}{to} balance'] - dic[f'{from_}{to} total_invested'])

            
        
        # Write the results into the db
        with engine.begin() as conn:
            conn.execute(text("INSERT INTO "+from_+to+"_results(period, position, balance, profitloss, status) VALUES (:period, :position, :balance, :profitloss, :status)"),[{"period": hours_past, "position": dic[f'{from_}{to} position'], "balance": dic[f'{from_}{to} balance'], "profitloss": dic[f'{from_}{to} profit_loss'], "status": dic[f'{from_}{to} trade_status']}])
                    
        
        
# This main function repeatedly calls the polygon api every 1 seconds for 24 hours 
# and stores the results.
def main(currency_pairs):
    # The api key given by the professor
    key = "beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq"
    
    # Number of list iterations - each one should last about 1 second
    count = 0
    agg_count = 0
    hour_count = 0
    hours_past = 0
    
    # Create a dictionary of variables that will act as local storage for the various currency paris
    dic = {}
    
    for currency in currency_pairs:
        # Set the input variables to the API
        from_ = currency[0]
        to = currency[1]
        
        #Initialise the variables that we will need
        dic[f'{from_}{to} maximum'] = float('-inf')
        dic[f'{from_}{to} minimum'] = float('inf')
        dic[f'{from_}{to} prices'] = []
        dic[f'{from_}{to} running_total'] = 0
        dic[f'{from_}{to} cross_count'] = 0
        dic[f'{from_}{to} period'] = 1
        dic[f'{from_}{to} keltner_upper_bands'] = []
        dic[f'{from_}{to} keltner_lower_bands'] = []
        dic[f'{from_}{to} avg_price'] = None
        dic[f'{from_}{to} last_price'] = None
        dic[f'{from_}{to} upper_count'] = 0
        dic[f'{from_}{to} lower_count'] = 0
        dic[f'{from_}{to} fd'] = 0
        dic[f'{from_}{to} old_mean'] = 0
        dic[f'{from_}{to} mean'] = 0
        dic[f'{from_}{to} return'] = 0
        # variables that will keep track of investments
        dic[f'{from_}{to} balance'] = 0       # we will initialise this to 100
        dic[f'{from_}{to} profit_loss'] = 0
        dic[f'{from_}{to} total_invested'] = 100
        dic[f'{from_}{to} number_of_hours'] = 0
        dic[f'{from_}{to} trade_status'] = '' # live or exited
        dic[f'{from_}{to} position'] = ''     # long or short
        
    
    
    
    # Create an engine to connect to the database; setting echo to false should stop it from logging in std.out
    engine = create_engine("sqlite+pysqlite:///sqlite/final.db", echo=False, future=True)
    
    
   
        
    # Create the needed tables in the database
    initialize_raw_data_tables(engine,currency_pairs)
    initialize_aggregated_tables(engine,currency_pairs)
    intialize_ts_tables(engine,currency_pairs)
    intialize_results_tables(engine,currency_pairs)
    
    
    
    # Start trading!
    initialise_trading(engine, currency_pairs, dic)
    
    # Open a RESTClient for making the api calls
    with RESTClient(key) as client:
        # Loop that runs until the total duration of the program hits 24 hours. 
        while count < 86400: # 86400 seconds = 24 hours
            
            
            # Make a check to see if 6 minutes has been reached or not
            if agg_count == 360:
                
                
                # Aggregate the data and clear the raw data tables
                aggregate_raw_data_tables(engine,currency_pairs)
                reset_raw_data_tables(engine,currency_pairs)
                agg_count = 0
            
            
            # Every ten hours run the trailing stop strategy (3600 seconds)
            if hour_count == 3600:

                
                # Run the stop loss strategy
                stop_loss_strategy(engine, currency_pairs, dic, hours_past)
                
                 # Increment hours that have past
                hours_past += 1

                # Reset hour count
                hour_count = 0
                
               

            # Only call the api every 1 second, so wait here for 0.75 seconds, because the 
            # code takes about .15 seconds to run
            time.sleep(0.75)
            
            # Increment the counters
            count += 1
            agg_count += 1
            hour_count += 1

            # Loop through each currency pair
            for currency in currency_pairs:
                # Set the input variables to the API
                from_ = currency[0]
                to = currency[1]
                
                if agg_count == 360:
                
                    # Every six minutes...
                    
                    # Calculate the mean
                    prices_arr = np.array(dic[f'{from_}{to} prices'])
                    dic[f'{from_}{to} running_total'] = prices_arr.sum()
                    dic[f'{from_}{to} mean'] = dic[f'{from_}{to} running_total'] / len(prices_arr)
                    
                    # Calculate the return
                    if  dic[f'{from_}{to} period'] > 1:
                        dic[f'{from_}{to} return'] = (dic[f'{from_}{to} mean'] - dic[f'{from_}{to} old_mean']) / dic[f'{from_}{to} old_mean']
                    else:
                        dic[f'{from_}{to} return'] = 0
                        
                        
                    # Assign the previous mean to the old_mean
                    dic[f'{from_}{to} old_mean'] = dic[f'{from_}{to} mean']

                    
                    # Reset values
                    dic[f'{from_}{to} prices'] = []
                    dic[f'{from_}{to} running_total'] = 0
    

                
                    # Write the volatility info to database
                
                    with engine.begin() as conn:
                        conn.execute(text("INSERT INTO "+from_+to+"_ts(period, maximum, minimum, mean, volatilty, fd, return) VALUES (:period, :maximum, :minimum, :mean, :volatilty, :fd, :return)"),[{"period": dic[f'{from_}{to} period'], "maximum": dic[f'{from_}{to} maximum'], "minimum": dic[f'{from_}{to} minimum'], "mean": dic[f'{from_}{to} mean'], "volatilty": dic[f'{from_}{to} vol'], "fd": dic[f'{from_}{to} fd'], "return": dic[f'{from_}{to} return']}])
                    
                    # Reset the currency specific variables
                    dic[f'{from_}{to} maximum'] = float('-inf')
                    dic[f'{from_}{to} minimum'] = float('inf')
                    dic[f'{from_}{to} cross_count'] = 0 
                    dic[f'{from_}{to} period'] +=1

                    dic[f'{from_}{to} keltner_upper_bands'] = []
                    dic[f'{from_}{to} keltner_lower_bands'] = []
                    dic[f'{from_}{to} upper_count'] = 0
                    dic[f'{from_}{to} lower_count'] = 0

                    

                    # If the first period has passed, calculate the keltner bands
                    if dic[f'{from_}{to} period'] > 1:

                    # Create 100 upper Keltner bands
                        for num in range(100):
                            calc = dic[f'{from_}{to} mean'] + num*0.025*dic[f'{from_}{to} vol']
                            dic[f'{from_}{to} keltner_upper_bands'].append(calc)
                            
                    # Create 100 lower Keltner bands
                        for num in range(100):
                            calc = dic[f'{from_}{to} mean'] - num*0.025*dic[f'{from_}{to} vol']
                            dic[f'{from_}{to} keltner_lower_bands'].append(calc)

                        
                

                # Call the API with the required parameters
                try:
                    resp = client.forex_currencies_real_time_currency_conversion(from_, to, amount=100, precision=2)
                except:
                    continue

                # This gets the Last Trade object defined in the API Resource
                last_trade = resp.last

                # Format the timestamp from the result
                dt = ts_to_datetime(last_trade["timestamp"])

                # Get the current time and format it
                insert_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                
                # Assign the old average price to the last price
                dic[f'{from_}{to} last_price'] = dic[f'{from_}{to} avg_price']
                
                
                # Calculate the new average price by taking the average of the bid and ask prices
                avg_price = (last_trade['bid'] + last_trade['ask'])/2
                dic[f'{from_}{to} avg_price'] = (last_trade['bid'] + last_trade['ask'])/2
                
                # Calculate the max price in the past six minutes
                if dic[f'{from_}{to} avg_price'] > dic[f'{from_}{to} maximum']:
                    dic[f'{from_}{to} maximum'] = dic[f'{from_}{to} avg_price']
                            
                # Calculate the min price in the last six minutes
                if dic[f'{from_}{to} avg_price'] < dic[f'{from_}{to} minimum']:
                    dic[f'{from_}{to} minimum'] = dic[f'{from_}{to} avg_price']
                
                # Calculate the volatility over the last six minutes
                dic[f'{from_}{to} vol'] = dic[f'{from_}{to} maximum'] - dic[f'{from_}{to} minimum']
                
                
                # Calculate the fractal dimension
                # For each new price we want to know how many bands have been crossed. 
        
                upper = np.array(dic[f'{from_}{to} keltner_upper_bands'])
                lower = np.array(dic[f'{from_}{to} keltner_lower_bands'])
                
                # How many numbers in the keltner band are greater than the old price and less than the new price
                if dic[f'{from_}{to} last_price'] is not None:
                    dic[f'{from_}{to} upper_count'] = ((dic[f'{from_}{to} last_price'] < upper) & (upper < dic[f'{from_}{to} avg_price'])).sum()

                # How many numbers in the keltner band are less than the old price and greater than the new price
                if dic[f'{from_}{to} last_price'] is not None:
                    dic[f'{from_}{to} lower_count'] = ((dic[f'{from_}{to} last_price'] > lower) & (lower > dic[f'{from_}{to} avg_price'])).sum()

                # Add the above counts from upper and lower bands together
                dic[f'{from_}{to} cross'] = dic[f'{from_}{to} upper_count'] + dic[f'{from_}{to} lower_count']
                
                # Add the total to the running total of crosses over the six minute period
                dic[f'{from_}{to} cross_count'] = dic[f'{from_}{to} cross_count'] +  dic[f'{from_}{to} cross']
                
                # Divide the cross_count by the volatility in order to calculte the fractal dimenstion
                if  dic[f'{from_}{to} vol'] != 0:
                     dic[f'{from_}{to} fd'] = dic[f'{from_}{to} cross_count'] /  dic[f'{from_}{to} vol']  
                
                # Keep trak of prices over last 6 minutes
                (dic[f'{from_}{to} prices']).append(dic[f'{from_}{to} avg_price'])
                
                
                # Write the data to the SQLite database, raw data tables
                with engine.begin() as conn:
                    conn.execute(text("INSERT INTO "+from_+to+"_raw(ticktime, fxrate, inserttime) VALUES (:ticktime, :fxrate, :inserttime)"),[{"ticktime": dt, "fxrate": avg_price, "inserttime": insert_time}])
                
                
                    

In [14]:
# A dictionary defining the set of currency pairs we will be pulling data for
currency_pairs = [["AUD","USD",[],portfolio("AUD","USD")],
                  ["GBP","EUR",[],portfolio("GBP","EUR")],
                  ["USD","CAD",[],portfolio("USD","CAD")],
                  ["USD","JPY",[],portfolio("USD","JPY")],
                  ["USD","MXN",[],portfolio("USD","MXN")],
                  ["EUR","USD",[],portfolio("EUR","USD")],
                  ["USD","CNY",[],portfolio("USD","CNY")],
                  ["USD","CZK",[],portfolio("USD","CZK")],
                  ["USD","PLN",[],portfolio("USD","PLN")],
                  ["USD","INR",[],portfolio("USD","INR")]]

# Run the main data collection loop
main(currency_pairs)

Running strategy
Running strategy
Running strategy
Running strategy
Running strategy
Running strategy
Running strategy
Running strategy
Running strategy
Running strategy
Running strategy


KeyboardInterrupt: 

# The following code blocks were used on historical data to fomulate a strategy

In [ ]:
# Historical data used the russian ruble, but the live data no longer uses it. So we define a class 
# for it here. 
# Define the USDRUB_return class - each instance will store one row from the dataframe
class USDRUB_return(object):
    # Variable to store the total number of instantiated objects in this class
    num = 0
    # Variable to store the running sum of the return
    run_sum = 0
    run_squared_sum = 0
    run_sum_of_std = 0
    last_price = -1
    
    # Init all the necessary variables when instantiating the class
    def __init__(self, tick_time, avg_price):
        
        # Store each column value into a variable in the class instance
        self.tick_time = tick_time
        #self.price = avg_price
        
        if USDRUB_return.last_price == -1:
            hist_return = float('NaN')
        else:
            hist_return = (avg_price - USDRUB_return.last_price) / USDRUB_return.last_price
        
        self.hist_return = hist_return
        if isnan(hist_return):
            USDRUB_return.run_sum = 0
        else:
            # Increment the counter
            if USDRUB_return.num < 5:
                USDRUB_return.num += 1
            USDRUB_return.run_sum += hist_return
        USDRUB_return.last_price = avg_price
        
    def add_to_running_squared_sum(self,avg):
        if isnan(self.hist_return) == False:
            USDRUB_return.run_squared_sum += (self.hist_return - avg)**2
    
    def get_avg(self,pop_value):
        if isnan(self.hist_return) == False:
            USDRUB_return.run_sum -= pop_value
            avg = USDRUB_return.run_sum/(USDRUB_return.num)
            self.avg_return = avg
            return avg
    
    def get_std(self):
        if isnan(self.hist_return) == False:
            std = sqrt(USDRUB_return.run_squared_sum/(USDRUB_return.num))
            self.std_return = std
            USDRUB_return.run_sum_of_std += std
            USDRUB_return.run_squared_sum = 0
            return std
    
    def get_avg_std(self,pop_value):
        if isnan(self.hist_return) == False:
            USDRUB_return.run_sum_of_std -= pop_value
            avg_std = USDRUB_return.run_sum_of_std/(USDRUB_return.num)
            self.avg_of_std_return = avg_std 
            return avg_std

In [ ]:
# This function is called every 6 minutes to aggregate the data, make the necessary calculations, 
# and make a decision about buying
def offline_aggregate_raw_data_tables(engine,currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            result = conn.execute(text("SELECT inserttime, avgfxrate FROM "+curr[0]+curr[1]+"_agg;"))
            for row in result:
                avg_price = row.avgfxrate
                last_date = row.inserttime
                
                # This calculates and stores the return values
                exec("curr[2].append("+curr[0]+curr[1]+"_return(last_date,avg_price))")
                #exec("print(\"The return for "+curr[0]+curr[1]+" is:"+str(curr[2][-1].hist_return)+" \")")

                if len(curr[2]) > 5:
                    try:
                        avg_pop_value = curr[2][-6].hist_return
                    except:
                        avg_pop_value = 0
                    if isnan(avg_pop_value) == True:
                        avg_pop_value = 0
                else:
                    avg_pop_value = 0
                # Calculate the average return value and print it/store it
                curr_avg = curr[2][-1].get_avg(avg_pop_value)
                #exec("print(\"The average return for "+curr[0]+curr[1]+" is:"+str(curr_avg)+" \")")

                # Now that we have the average return, loop through the last 5 rows in the list to start compiling the 
                # data needed to calculate the standard deviation
                for row in curr[2][-5:]:
                    row.add_to_running_squared_sum(curr_avg)

                # Calculate the standard dev using the avg
                curr_std = curr[2][-1].get_std()
                #exec("print(\"The standard deviation of the return for "+curr[0]+curr[1]+" is:"+str(curr_std)+" \")")

                # Calculate the average standard dev
                if len(curr[2]) > 5:
                    try:
                        pop_value = curr[2][-6].std_return
                    except:
                        pop_value = 0
                else:
                    pop_value = 0
                curr_avg_std = curr[2][-1].get_avg_std(pop_value)
                #exec("print(\"The average standard deviation of the return for "+curr[0]+curr[1]+" is:"+str(curr_avg_std)+" \")")

                # -------------------Investment Strategy-----------------------------------------------
                try:
                    return_value = curr[2][-1].hist_return
                except:
                    return_value = 0
                if isnan(return_value) == True:
                    return_value = 0

                try:
                    return_value_1 = curr[2][-2].hist_return
                except:
                    return_value_1 = 0
                if isnan(return_value_1) == True:
                    return_value_1 = 0

                try:
                    return_value_2 = curr[2][-3].hist_return
                except:
                    return_value_2 = 0
                if isnan(return_value_2) == True:
                    return_value_2 = 0
                
                try:
                    upp_band = curr[2][-1].avg_return + (1.5 * curr[2][-1].std_return)
                    if return_value >= upp_band and curr[3].Prev_Action_was_Buy == True and return_value != 0: #  (return_value > 0) and (return_value_1 > 0) and   
                        curr[3].sell_curr(avg_price)
                except:
                    pass
                
                try:
                    loww_band = curr[2][-1].avg_return - (1.5 * curr[2][-1].std_return)
                    if return_value <= loww_band and curr[3].Prev_Action_was_Buy == False and return_value != 0: # and  (return_value < 0) and (return_value_1 < 0)
                        curr[3].buy_curr(avg_price)
                except:
                    pass

# A dictionary defining the set of currency pairs we will be pulling data for
currency_pairs = [["AUD","USD",[],portfolio("AUD","USD")],
                  ["GBP","EUR",[],portfolio("GBP","EUR")],
                  ["USD","CAD",[],portfolio("USD","CAD")],
                  ["USD","JPY",[],portfolio("USD","JPY")],
                  ["USD","MXN",[],portfolio("USD","MXN")],
                  ["EUR","USD",[],portfolio("EUR","USD")],
                  ["USD","RUB",[],portfolio("USD","RUB")],
                  ["USD","CZK",[],portfolio("USD","CZK")],
                  ["USD","PLN",[],portfolio("USD","PLN")],
                  ["USD","INR",[],portfolio("USD","INR")]]

# Function to run the necessary testing on offline data
def main_offline(currency_pairs):
    # Create an engine to connect to the database
    engine = create_engine("sqlite+pysqlite:///sqlite/offline.db", echo=False, future=True)
    offline_aggregate_raw_data_tables(engine,currency_pairs)            

In [ ]:
main_offline(currency_pairs)

In [ ]:
print(currency_pairs[0][2][100])

print(currency_pairs[0][2][100].hist_return)


In [ ]:
# This section plots the historical returns with their corressponding bollinger bands. It also
# prints the total profits/losses for each currency pair, and the total across all currency pairs. 

# Create a subplot
fig, axs = plt.subplots(10,figsize=(10,40))
fig.tight_layout()

# Variable to keep track of the total profit across currency pairs. 
tot_profit = 0

# Loop through the currency pairs
for ind, currency in enumerate(currency_pairs):
    
    from_ = currency[0]
    to = currency[1]
    
    # The sublists in the following list represent each of the following:
    # hist_return, avg_return, std_return, avg_of_std_return, upper bollinger, lower bollinger
    returns_array = [[],[],[],[],[],[]]
    
    # Extract the data from the classes and put it into a single list for plotting
    for row in currency[2]:
        returns_array[0].append(row.hist_return)
        try:
            returns_array[1].append(row.avg_return)
        except:
            returns_array[1].append(0)
        try:
            returns_array[2].append(row.std_return)
        except:
            returns_array[2].append(0)
        try:
            returns_array[3].append(row.avg_of_std_return)
        except:
            returns_array[3].append(0)
        try:
            returns_array[4].append(row.avg_return + (1.5 * row.std_return))
        except:
            returns_array[4].append(0)
        try:
            returns_array[5].append(row.avg_return - (1.5 * row.std_return))
        except:
            returns_array[5].append(0)
            
    print("The profit/losses for "+from_+to+" calculated with numpy is: %f" % (currency[3].amount -1))
    tot_profit += currency[3].amount - 1
    
    # Plot the line graphs with bollinger bands using the propper formatting
    axs[ind].plot(range(0,len(returns_array[0])),returns_array[0]) # plot the historical returns
    axs[ind].plot(range(0,len(returns_array[4])),returns_array[4]) # plot the upper bollinger band for returns
    axs[ind].plot(range(0,len(returns_array[5])),returns_array[5]) # plot the lower bollinger band for returns
    axs[ind].set(xlabel='Time',ylabel='Return')
    axs[ind].set_title(from_+to+'  Returns Over Time')
    
# Extra formatting to make sure the axis labels do not overlap the titles
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.4, 
                    hspace=0.4)

print("Total profit across currencies is: %f" % tot_profit)